In [ ]:
import re
import pandas as pd

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, pipeline

model_name = "cardiffnlp/twitter-roberta-base-sentiment"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model_name, device=0)
# config = AutoConfig.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

label_map = {
    'LABEL_0': 'NEGATIVE',
    'LABEL_1': 'NEUTRAL',
    'LABEL_2': 'POSITIVE'
}

from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [ ]:
def remove_html(text):
    clean = re.sub(r'<[^>]+>', ' ', text)
    return re.sub(r'\s+', ' ', clean).strip()

def clean_review(text):
    if pd.isna(text):
        return None
    text = remove_html(text)
    return text

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "error"


def classify_sentiment(text):
    try:
        return label_map[classifier(text[:514])[0]['label']]
    except Exception as e:
        print(f"Error processing: {text[:60]}... -> {e}")
        return "error"

* PREPROCESSING

In [ ]:
df = pd.read_csv("data/reviews.csv")
df.dropna(subset=['comments'], inplace=True)

df['comments'] = df['comments'].apply(clean_review)

df = df[df['comments'].apply(detect_language) == 'en']

SENTIMENT ANALYSIS

In [ ]:
df['sentiment'] = df['comments'].apply(classify_sentiment)

In [ ]:
df[df['sentiment'] != 'NEUTRAL'].to_csv('data/sentiment_reviews.csv', index=False)
df[df['sentiment'] == 'NEGATIVE'][['comments', 'sentiment']].to_csv('data/negative_reviews.csv', index=False)
df[df['sentiment'] == 'POSITIVE'][['comments', 'sentiment']].to_csv('data/positive_reviews.csv', index=False)

print("Sentiment classification completed and saved to sentiment_reviews.csv")
print("Negative reviews saved to negative_reviews.csv size:", len(df[df['sentiment'] == 'NEGATIVE'].index))
print("Positive reviews saved to positive_reviews.csv size:", len(df[df['sentiment'] == 'POSITIVE'].index))